In [40]:
import re
from collections import Counter
import pymorphy2
from tqdm import tqdm
import os
import pandas as pd

In [109]:
marked_path = '/home/nst/mount/data/share/yd/popular_science_texts_store/ner_markup/final_markup/'

In [3]:
def slurp(path):
    with open(path, 'r') as file_object:
        return file_object.read()

In [4]:
def extract_ne(text):
    pattern = re.compile(r'&(.*?)!&')
    nes = re.findall(pattern, text)
    lemmas = []
    for ne in nes:
        divided = ne.split()
        lemma = [morph.parse(word)[0].normal_form for word in divided]
        lemma = ' '.join(lemma)
        lemmas.append(lemma)
    return lemmas

In [4]:
text = 'Есть очень неудачные способы разведения рыб, однако если посмотреть на &Энштейна!& лучшие практики, то они намного меньше вредят природе, чем любое производство на суше», — отметил декан Школы экологии и управления Брена Калифорнийского университета &Стив Гейнс!& в Санта Барбаре'

In [74]:
def range_ne(texts_path, top_number):
    nes = []
    texts = []
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            marked_text = slurp(input_path)
            texts.append(marked_text)
    for text in tqdm(texts):
        found_ne = extract_ne(text)
        #print(found_ne)
        nes.extend(found_ne)
    ne_dict = Counter(nes)
    ranged_ne = list(sorted(ne_dict, key = lambda x : x[1], reverse=True))
    print(top_number, "most cited scholars are:\n", ranged_ne[:top_number]) 
    return ranged_ne

## Get NE context

In [147]:
def preprocess(texts):
    pattern = re.compile(r'[A-Za-zА-ЯЁа-яё\-\.]+')
    clean_texts = []
    for text in tqdm(texts):
        split_text = re.findall(pattern, text)
        joined_text = ' '.join(split_text)
        clean_texts.append(joined_text)
    preproc_texts = pd.DataFrame(clean_texts, columns=['preprocessed_texts'])
    return preproc_texts

In [42]:
texts = [text, text_1]
preproc = preprocess(texts)

100%|██████████| 2/2 [00:00<00:00, 6398.63it/s]


In [46]:
preproc['marked_texts']=texts

In [47]:
preproc

,preprocessed_texts,marked_texts
0,Есть очень неудачные способы разведения рыб од...,"Есть очень неудачные способы разведения рыб, о..."
1,Я позволю Девантье продолжить Он начинает со с...,Я позволю Девантье продолжить. Он начинает со ...


In [163]:
def extract_entities_indices(clean_text, marked_text):
    indices = []
    entity_index_left = None
    entity_index_right = None
    pattern = re.compile(r'&(.*?)!&')
    nes = re.findall(pattern, marked_text)
    print(nes)
    for name in nes:
        name = name.split()
        #метод индекс дает индекс первого символа строки
       # try:
        if len(name)==1:
            name = ''.join(name)
            entity_index_left = clean_text.index(name) #индекс начала строки имени
            entity_index_right = clean_text.index(name) + len(name) #индекс конца строки
                                                                        #имени
        else:
            entity_index_left = clean_text.index(name[0])
            entity_index_right = clean_text.index(name[-1]) + len(name[-1])
        index_pair = (entity_index_left, entity_index_right)
        indices.append(index_pair)
    return indices
      #  except ValueError:
      #      if len(name)==1:
      #          name = ''.join(name)
      #          name = name[:-1]
      #          entity_index_left = clean_text.index(name)
      #          entity_index_right = clean_text.index(name) + len(name)
      #      else:
      #          name_first = name[0]
      #          name_first = name_first[:-1]
      #          name_last = name[-1]
      #          name_last = name_last[:-1]
      #          
      #          entity_index_left = clean_text.index(name_first)
      #          entity_index_right = clean_text.index(name_last) + len(name_last)
      #      index_pair = (entity_index_left, entity_index_right)
      #      indices.append(index_pair)

In [82]:
ind = extract_entities_indices(preproc.preprocessed_texts[1], preproc.marked_texts[1])

In [19]:
text_1 = 'Я позволю Девантье продолжить. Он начинает со ссылки на исследования &Флойда Тула!& (&Floyd Toole!&), своего бывшего коллеги, и &Шона Олива!& (&Sean Olive!&), уже нынешнего коллеги в Harman, которые когда-то работали в Национальном исследовательском совете Канады:'\
'В 1986 году &Флойд Тул!& и &Шон Олив!& опубликовали исследование о слышимости резонансных колебаний. Они выяснили, что слушатели наиболее чувствительны к низкодобротным (с высокой пропускной способностью) колебаниям. При правильных условиях можно было услышать среднечастотные пики всего в 0,3 децибел (дБ).'

In [123]:
def get_context(clean_text, entities_indices, window):
    #word_index = word_list.index('crisis')
    left = []
    names = []
    right = []
    for index_pair in entities_indices:
        left_index = index_pair[0]
        right_index = index_pair[1] + 1
        name = clean_text[left_index:right_index]
        names.append(name)
        left_context = clean_text[:left_index].split()
        if len(left_context) < window:
            left_context = clean_text[:left_index]
        else:
            left_context = ' '.join(left_context[-window:])
        left.append(left_context)    
        right_context = clean_text[right_index:].split()
        if len(right_context) < window:
            right_context = clean_text[right_index:]
        else:
            right_context = ' '.join(right_context[:window+1])
        right.append(right_context)
    return left, names, right

In [83]:
left, names, right = get_context(preproc.preprocessed_texts[1], ind, 3)

In [89]:
left

['ссылки на исследования',
 'исследования Флойда Тула',
 'бывшего коллеги и',
 'и Шона Олива',
 'ссылки на исследования',
 'бывшего коллеги и']

1. entry values: clean text, marked_text, window
2. Make a list of indices for every text

In [95]:
def get_all_contexts(clean_texts, marked_texts, window):
    indices_list = [extract_entities_indices(clean_text, marked_text)
              for clean_text, marked_text in zip(clean_texts, marked_texts)]
    lefts_list = []
    names_list = []
    rights_list = []
    for text, indices in zip(clean_texts, indices_list):
        left, names, right = get_context(text, indices, window)
        lefts_list.extend(left)
        names_list.extend(names)
        rights_list.extend(right)
    dataframe = pd.DataFrame(lefts_list, columns=['left_context'])
    dataframe['named_entities'] = names_list
    dataframe['right_context'] = rights_list
    return dataframe

In [135]:
get_all_contexts(preproc.preprocessed_texts, preproc.marked_texts, 5)

['Энштейна', 'Стив Гейнс']
['Флойда Тула', 'Floyd Toole', 'Шона Олива', 'Sean Olive', 'Флойд Тул', 'Шон Олив']


,left_context,named_entities,right_context
0,рыб однако если посмотреть на,Энштейна,лучшие практики то они намного меньше
1,и управления Брена Калифорнийского университета,Стив Гейнс,в Санта Барбаре
2,начинает со ссылки на исследования,Флойда Тула,Floyd Toole своего бывшего коллеги и
3,ссылки на исследования Флойда Тула,Floyd Toole,своего бывшего коллеги и Шона Олива
4,Toole своего бывшего коллеги и,Шона Олива,Sean Olive уже нынешнего коллеги в
5,бывшего коллеги и Шона Олива,Sean Olive,уже нынешнего коллеги в Harman которые
6,начинает со ссылки на исследования,Флойда Тула,Floyd Toole своего бывшего коллеги и
7,Toole своего бывшего коллеги и,Шона Олива,Sean Olive уже нынешнего коллеги в


## Make Dataframe and get all NE contexts

### Marked & Preprocessed DataFrame

In [138]:
def make_texts_df(texts_path):
    marked_texts = []
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            marked_text = slurp(input_path)
            marked_texts.append(marked_text)
    marked_df = pd.DataFrame(marked_texts, columns=['marked_texts'])
    return marked_df

In [148]:
marked_df = make_texts_df(marked_path)

In [149]:
preprocessed_texts = preprocess(marked_df.marked_texts)

100%|██████████| 165/165 [00:00<00:00, 956.42it/s]


In [152]:
texts_df = marked_df.join(preprocessed_texts) ;

In [154]:
texts_df.preprocessed_texts[2]

'Радиопульсары одно из наблюдательных проявлений нейтронных звезд источники пульсирующего радиоизлучения с периодами от нескольких миллисекунд до секунд. Они были открыты лет назад. Пожалуй широкомасштабное наблюдательное изучение нейтронных звёзд началось именно с открытия радиопульсаров в году в Маллардской обсерватории в Великобритании. Это было сделано во время исследований мерцаний радиоисточников. Первооткрывателями этих объектов можно назвать тогдашнюю аспирантку Джоселин Белл и ее руководителя Энтони Хьюиша . Причем понимание того с источниками какой природы они имеют дело пришло не сразу. Хьюиш если верить пересказам сначала не вполне поверил что первый открытый объект был именно космического природного происхождения и не был артефактом. И даже когда таких источников обнаружилось несколько им было присвоено обозначение LGM Little Green Men маленькие зелёные человечки . То есть был очень большой соблазн из-за строгой периодичности излучения говорить о том что это сигналы разумн

### Get all contexts from marked texts with window 5

In [164]:
contexts_df = get_all_contexts(texts_df.preprocessed_texts, texts_df.marked_texts, 5)

['Ричарда Фейнмана', 'Гердом Биннигом', 'Генрихом Рорером', 'Дональдом Эйглером', 'Эрхардом Швейцером', 'Джозефа Стросцио', 'Дональд Эйглер', 'Первое', 'Капоши']
['Первое', 'Андрей Анатольевич Зализняк', 'Зализняка', 'Ирины Левонтиной', 'Максима Кронгауза', 'Кронгауз', 'Панова', 'Маркса', 'Эйнштейна', 'Логунове', 'Зализняк', 'Зализняка']
['Джоселин Белл', 'Энтони Хьюиша', 'Хьюиш', 'Э. Хьюишу', 'Джоселин Белл']
['Скажем', 'Ричарда Ф. Фейнмана', 'Фейнман', 'Менделеева', 'Максвелл', 'Эйнштейн']
['Ричарда Форти', 'Форти', 'Гарри Уиттингтон', 'Дерек Бриггс', 'Саймон Конвей Моррис', 'Уолкотта', 'Гарри Уиттингтон', 'Саймона Конвея Морриса', 'Саймону Конвею Моррису', 'Саймону Конвею Моррису', 'Гарри Уиттингтона', 'Саймон Конвей Моррис', 'Дерек Бриггс', 'Дэвид Брутон', 'Крис Хьюз', 'Кембридж', 'Дереком Бриггсом', 'Стивеном Гулдом', 'Дреком Бриггсом', 'Дэвид Суоффорд', 'Суоффорд', 'Хокинг', 'Мэтью Уилз', 'Ричарда Фейнмана', 'Стивена Хокинга', 'Грег Эджкомб']
['Бориса Родомана', 'Борис Родоман', 

ValueError: substring not found

In [145]:
'K. Wolf-Schwenniger'.split()

['K.', 'Wolf-Schwenniger']